**TP5:**
1.Importa las librerías necesarias:

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


2. Define los parámetros necesarios para el detector de fondo naive:

In [2]:
N = 50 # cantidad de frames utilizados para la estimación
intervalo = 100 # intervalo de tiempo para recalcular el fondo
alpha = 0.5 # factor de aprendizaje para la mediana


3.Carga el video y extrae los frames:

In [3]:
cap = cv2.VideoCapture('/content/slow_traffic_small.mp4')
frames = []
while True:
  ret, frame = cap.read()
  if not ret:
    break
  frames.append(frame)


4. Calcula el fondo utilizando la mediana de los primeros N frames:

In [4]:
fondo = np.median(frames[:N], axis=0).astype(np.uint8)


5. Recorre los frames restantes y calcula la máscara de foreground utilizando la diferencia absoluta entre el frame y el fondo:

In [5]:
mascaras = []
for i in range(N, len(frames)):
  frame = frames[i]
  mascara = cv2.absdiff(frame, fondo)
  mascara = cv2.cvtColor(mascara, cv2.COLOR_BGR2GRAY)
  mascara = cv2.threshold(mascara, 25, 255, cv2.THRESH_BINARY)[1]
  mascaras.append(mascara)

  if i % intervalo == 0:
    fondo = np.median(frames[i-N:i], axis=0).astype(np.uint8)


6. Aplica las mascaras a los frames para segmentar los objetos en movimiento:

In [ ]:
for i in range(len(frames)):
  frame = frames[i]
  mascara = mascaras[i-N]
  resultado = cv2.bitwise_and(frame, frame, mask=mascara)

  # Mostrar el resultado
  from google.colab.patches import cv2_imshow
# ...
  cv2_imshow(frame)
  if cv2.waitKey(25) & 0xFF == ord('q'):
    break



CONCLUSION:
El método que utilicé fue el detector de fondo naive basado en la mediana, que estima el fondo utilizando una ventana deslizante de N frames y recalculando el fondo cada cierto intervalo de tiempo. Este método es simple y computacionalmente eficiente, pero puede tener problemas cuando hay cambios significativos en la iluminación o cuando hay objetos en movimiento continuo.

Por otro lado, el método de backprojection utiliza una imagen de histograma de color para detectar objetos en movimiento basados en la comparación de la distribución de color del objeto de interés con la distribución de color del fondo. Es efectivo en situaciones donde el objeto de interés tiene un color distintivo, pero puede ser menos efectivo en situaciones de iluminación cambiante o cuando el objeto de interés está parcialmente oculto.

Finalmente, el algoritmo de meanshift utiliza una ventana deslizante para buscar el objeto de interés en cada frame, y ajusta la posición de la ventana para maximizar la similitud entre la distribución de color del objeto y la del fondo. Este método es robusto ante cambios en la iluminación y cuando hay objetos en movimiento continuo, pero puede ser computacionalmente costoso en aplicaciones en tiempo real.